Cell 1 — Load dataset


In [2]:
import pandas as pd

df = pd.read_csv(r"A:\Projects\Finance\Smart_Personal_Expense_Analyzer-main\python\ai\expense_nlp_master_2500_v2.csv")
df.head()


,text_raw,text_clean,intent,category,emotion,emotion_score,confidence_label,source,language,multi_category,alt_category,created_at
0,ate outside today,ate outside today,food_order,groceries,casual,0.20,0.96,manual,hinglish,False,NaN,2025-03-23
1,netflix renewal,netflix renewal,subscription,subscription,care,0.28,0.88,manual,english,False,NaN,2024-06-19
2,junk food again 😐,junk food again,food_order,groceries,casual,0.23,0.82,sms,english,False,NaN,2025-01-09
3,late night food craving,late night food craving,food_order,dining,stress,-0.49,0.82,sms,hinglish,False,NaN,2025-04-11
4,engine oil change,engine oil change,vehicle_maintenance,vehicle_repair,neutral,0.11,0.86,sms,english,False,NaN,2024-11-07


Cell 2 — Load embedding model

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
print("Loaded successfully")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\raj47\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\raj47\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded successfully


Cell 3 — Generate embeddings

In [6]:
texts = df["text_raw"].tolist()
embeddings = model.encode(texts, show_progress_bar=True)

len(embeddings), embeddings[0].shape


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

(2500, (384,))

Cell 4 — Save embeddings 

In [7]:
import pickle

with open("ai_text_embeddings.pkl", "wb") as f:
    pickle.dump({
        "texts": texts,
        "embeddings": embeddings
    }, f)


Category centroid idea

In [8]:
import numpy as np

category_centroids = {}

for category in df["category"].unique():
    idx = df[df["category"] == category].index
    category_centroids[category] = np.mean(
        embeddings[idx], axis=0
    )


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

test_text = "fuel refill for bike"
test_emb = model.encode([test_text])[0]

scores = {
    cat: cosine_similarity(
        [test_emb], [centroid]
    )[0][0]
    for cat, centroid in category_centroids.items()
}

sorted(scores.items(), key=lambda x: x[1], reverse=True)


[('fuel', 0.5510591),
 ('cab', 0.5350437),
 ('vehicle_repair', 0.4155302),
 ('electricity', 0.15442052),
 ('mobile_bill', 0.1538826),
 ('rent', 0.13864444),
 ('subscription', 0.053070426),
 ('shopping', 0.041758366),
 ('pet_supplies', 0.031875353),
 ('medical', 0.013104892),
 ('education', 0.009741978),
 ('groceries', 0.009672588),
 ('dining', 0.0035338737),
 ('entertainment', -0.016770642)]

In [10]:
import pickle

with open("category_centroids.pkl", "wb") as f:
    pickle.dump(category_centroids, f)


Cell 1 — Prepare data

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X = embeddings
y = df["intent"]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)


Cell 2 — Train model

In [12]:
from sklearn.linear_model import LogisticRegression

intent_model = LogisticRegression(
    max_iter=1000,
    multi_class="auto"
)

intent_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

Cell 3 — Evaluate

In [13]:
from sklearn.metrics import classification_report

y_pred = intent_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


                     precision    recall  f1-score   support

       bill_payment       1.00      1.00      1.00        43
            commute       1.00      1.00      1.00        40
          education       1.00      1.00      1.00        54
      entertainment       1.00      1.00      1.00        56
         food_order       1.00      1.00      1.00        57
             health       1.00      1.00      1.00        48
           pet_care       1.00      1.00      1.00        53
           shopping       1.00      1.00      1.00        49
       subscription       1.00      1.00      1.00        61
vehicle_maintenance       1.00      1.00      1.00        39

           accuracy                           1.00       500
          macro avg       1.00      1.00      1.00       500
       weighted avg       1.00      1.00      1.00       500



Cell 4 — Save model + encoder

In [14]:
import pickle

with open("intent_classifier.pkl", "wb") as f:
    pickle.dump(intent_model, f)

with open("intent_label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


Cell 1 — Prepare emotion data

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = embeddings                 # reuse same embeddings
y = df["emotion"]

emotion_encoder = LabelEncoder()
y_encoded = emotion_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)


Cell 2 — Train emotion model

In [16]:
from sklearn.linear_model import LogisticRegression

emotion_model = LogisticRegression(
    max_iter=1000,
    multi_class="auto"
)

emotion_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

Cell 3 — Evaluate

In [17]:
from sklearn.metrics import classification_report

y_pred = emotion_model.predict(X_test)
print(
    classification_report(
        y_test, y_pred,
        target_names=emotion_encoder.classes_
    )
)


              precision    recall  f1-score   support

     anxiety       0.09      0.14      0.11        56
        care       0.13      0.26      0.18        65
      casual       0.23      0.16      0.19        58
       happy       0.12      0.11      0.12        70
   impulsive       0.09      0.11      0.10        56
     neutral       0.14      0.10      0.11        63
      regret       0.07      0.03      0.04        69
      stress       0.18      0.10      0.12        63

    accuracy                           0.12       500
   macro avg       0.13      0.13      0.12       500
weighted avg       0.13      0.12      0.12       500



Cell 4 — Save model + encoder

In [18]:
import pickle

with open("emotion_classifier.pkl", "wb") as f:
    pickle.dump(emotion_model, f)

with open("emotion_label_encoder.pkl", "wb") as f:
    pickle.dump(emotion_encoder, f)


Cell 5 — Save emotion centroids

In [19]:
import numpy as np

emotion_centroids = {}

for emotion in df["emotion"].unique():
    idx = df[df["emotion"] == emotion].index
    emotion_centroids[emotion] = embeddings[idx].mean(axis=0)

with open("emotion_centroids.pkl", "wb") as f:
    pickle.dump(emotion_centroids, f)


Cell 1 — Prepare behavioral features


In [20]:
import numpy as np

# Example behavioral features (mock for now)
# In production, these come from user history
behavior_features = df[[
    "emotion_score",        # emotional intensity
    "confidence_label"      # NLP confidence
]].copy()

# Add synthetic normalized amount signal
np.random.seed(42)
behavior_features["amount_norm"] = np.random.normal(0.5, 0.15, len(df))


Cell 2 — Train Isolation Forest

In [21]:
from sklearn.ensemble import IsolationForest

anomaly_model = IsolationForest(
    n_estimators=100,
    contamination=0.1,   # 10% unusual behavior
    random_state=42
)

anomaly_model.fit(behavior_features)


IsolationForest(contamination=0.1, random_state=42)

Cell 3 — Test anomaly scores

In [22]:
scores = anomaly_model.decision_function(behavior_features)
flags = anomaly_model.predict(behavior_features)

df["anomaly_score"] = scores
df["is_anomaly"] = flags == -1

df[["emotion", "anomaly_score", "is_anomaly"]].head()


,emotion,anomaly_score,is_anomaly
0,casual,0.093525,False
1,care,0.115014,False
2,casual,-0.000744,True
3,stress,-0.029444,True
4,neutral,0.086941,False


Cell 4 — Save model

In [23]:
import pickle

with open("behavior_anomaly_model.pkl", "wb") as f:
    pickle.dump(anomaly_model, f)
